<a href="https://colab.research.google.com/github/Saroj4545/deepfake_detection/blob/main/DeepfakeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
download_script = """# -*- coding: utf-8 -*-
import argparse
import os
import urllib
import urllib.request
import tempfile
import time
import sys
import json
import random
from tqdm import tqdm
from os.path import join


# URLs and filenames
FILELIST_URL = 'misc/filelist.json'
DEEPFEAKES_DETECTION_URL = 'misc/deepfake_detection_filenames.json'
DEEPFAKES_MODEL_NAMES = ['decoder_A.h5', 'decoder_B.h5', 'encoder.h5',]

# Parameters
DATASETS = {
    'original_youtube_videos': 'misc/downloaded_youtube_videos.zip',
    'original_youtube_videos_info': 'misc/downloaded_youtube_videos_info.zip',
    'original': 'original_sequences/youtube',
    'DeepFakeDetection_original': 'original_sequences/actors',
    'Deepfakes': 'manipulated_sequences/Deepfakes',
    'DeepFakeDetection': 'manipulated_sequences/DeepFakeDetection',
    'Face2Face': 'manipulated_sequences/Face2Face',
    'FaceShifter': 'manipulated_sequences/FaceShifter',
    'FaceSwap': 'manipulated_sequences/FaceSwap',
    'NeuralTextures': 'manipulated_sequences/NeuralTextures'
    }
ALL_DATASETS = ['original', 'DeepFakeDetection_original', 'Deepfakes',
                'DeepFakeDetection', 'Face2Face', 'FaceShifter', 'FaceSwap',
                'NeuralTextures']
COMPRESSION = ['raw', 'c23', 'c40']
TYPE = ['videos', 'masks', 'models']
SERVERS = ['EU', 'EU2', 'CA']


def parse_args():
    parser = argparse.ArgumentParser(
        description='Downloads FaceForensics v2 public data release.',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    parser.add_argument('output_path', type=str, help='Output directory.')
    parser.add_argument('-d', '--dataset', type=str, default='all',
                        help='Which dataset to download, either pristine or '
                             'manipulated data or the downloaded youtube '
                             'videos.',
                        choices=list(DATASETS.keys()) + ['all']
                        )
    parser.add_argument('-c', '--compression', type=str, default='raw',
                        help='Which compression degree. All videos '
                             'have been generated with h264 with a varying '
                             'codec. Raw (c0) videos are lossless compressed.',
                        choices=COMPRESSION
                        )
    parser.add_argument('-t', '--type', type=str, default='videos',
                        help='Which file type, i.e. videos, masks, for our '
                             'manipulation methods, models, for Deepfakes.',
                        choices=TYPE
                        )
    parser.add_argument('-n', '--num_videos', type=int, default=None,
                        help='Select a number of videos number to '
                             "download if you don't want to download the full"
                             ' dataset.')
    parser.add_argument('--server', type=str, default='EU',
                        help='Server to download the data from. If you '
                             'encounter a slow download speed, consider '
                             'changing the server.',
                        choices=SERVERS
                        )
    args = parser.parse_args()

    # URLs
    server = args.server
    if server == 'EU':
        server_url = 'http://canis.vc.in.tum.de:8100/'
    elif server == 'EU2':
        server_url = 'http://kaldir.vc.in.tum.de/faceforensics/'
    elif server == 'CA':
        server_url = 'http://falas.cmpt.sfu.ca:8100/'
    else:
        raise Exception('Wrong server name. Choices: {}'.format(str(SERVERS)))
    args.tos_url = server_url + 'webpage/FaceForensics_TOS.pdf'
    args.base_url = server_url + 'v3/'
    args.deepfakes_model_url = server_url + 'v3/manipulated_sequences/' + \
                               'Deepfakes/models/'

    return args


def download_files(filenames, base_url, output_path, report_progress=True):
    os.makedirs(output_path, exist_ok=True)
    if report_progress:
        filenames = tqdm(filenames)
    for filename in filenames:
        download_file(base_url + filename, join(output_path, filename))


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\rProgress: %d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()


def download_file(url, out_file, report_progress=False):
    out_dir = os.path.dirname(out_file)
    if not os.path.isfile(out_file):
        fh, out_file_tmp = tempfile.mkstemp(dir=out_dir)
        f = os.fdopen(fh, 'w')
        f.close()
        if report_progress:
            urllib.request.urlretrieve(url, out_file_tmp,
                                       reporthook=reporthook)
        else:
            urllib.request.urlretrieve(url, out_file_tmp)
        os.rename(out_file_tmp, out_file)
    else:
        tqdm.write('WARNING: skipping download of existing file ' + out_file)


def main(args):
    # TOS
    print('By pressing any key to continue you confirm that you have agreed '\
          'to the FaceForensics terms of use as described at:')
    print(args.tos_url)
    print('***')
    print('Press any key to continue, or CTRL-C to exit.')
    _ = input('')

    # Extract arguments
    c_datasets = [args.dataset] if args.dataset != 'all' else ALL_DATASETS
    c_type = args.type
    c_compression = args.compression
    num_videos = args.num_videos
    output_path = args.output_path
    os.makedirs(output_path, exist_ok=True)

    # Check for special dataset cases
    for dataset in c_datasets:
        dataset_path = DATASETS[dataset]
        # Special cases
        if 'original_youtube_videos' in dataset:
            # Here we download the original youtube videos zip file
            print('Downloading original youtube videos.')
            if not 'info' in dataset_path:
                print('Please be patient, this may take a while (~40gb)')
                suffix = ''
            else:
            	suffix = 'info'
            download_file(args.base_url + '/' + dataset_path,
                          out_file=join(output_path,
                                        'downloaded_videos{}.zip'.format(
                                            suffix)),
                          report_progress=True)
            return

        # Else: regular datasets
        print('Downloading {} of dataset "{}"'.format(
            c_type, dataset_path
        ))

        # Get filelists and video lenghts list from server
        if 'DeepFakeDetection' in dataset_path or 'actors' in dataset_path:
        	filepaths = json.loads(urllib.request.urlopen(args.base_url + '/' +
                DEEPFEAKES_DETECTION_URL).read().decode("utf-8"))
        	if 'actors' in dataset_path:
        		filelist = filepaths['actors']
        	else:
        		filelist = filepaths['DeepFakesDetection']
        elif 'original' in dataset_path:
            # Load filelist from server
            file_pairs = json.loads(urllib.request.urlopen(args.base_url + '/' +
                FILELIST_URL).read().decode("utf-8"))
            filelist = []
            for pair in file_pairs:
            	filelist += pair
        else:
            # Load filelist from server
            file_pairs = json.loads(urllib.request.urlopen(args.base_url + '/' +
                FILELIST_URL).read().decode("utf-8"))
            # Get filelist
            filelist = []
            for pair in file_pairs:
                filelist.append('_'.join(pair))
                if c_type != 'models':
                    filelist.append('_'.join(pair[::-1]))
        # Maybe limit number of videos for download
        if num_videos is not None and num_videos > 0:
        	print('Downloading the first {} videos'.format(num_videos))
        	filelist = filelist[:num_videos]

        # Server and local paths
        dataset_videos_url = args.base_url + '{}/{}/{}/'.format(
            dataset_path, c_compression, c_type)
        dataset_mask_url = args.base_url + '{}/{}/videos/'.format(
            dataset_path, 'masks', c_type)

        if c_type == 'videos':
            dataset_output_path = join(output_path, dataset_path, c_compression,
                                       c_type)
            print('Output path: {}'.format(dataset_output_path))
            filelist = [filename + '.mp4' for filename in filelist]
            download_files(filelist, dataset_videos_url, dataset_output_path)
        elif c_type == 'masks':
            dataset_output_path = join(output_path, dataset_path, c_type,
                                       'videos')
            print('Output path: {}'.format(dataset_output_path))
            if 'original' in dataset:
                if args.dataset != 'all':
                    print('Only videos available for original data. Aborting.')
                    return
                else:
                    print('Only videos available for original data. '
                          'Skipping original.\n')
                    continue
            if 'FaceShifter' in dataset:
                print('Masks not available for FaceShifter. Aborting.')
                return
            filelist = [filename + '.mp4' for filename in filelist]
            download_files(filelist, dataset_mask_url, dataset_output_path)

        # Else: models for deepfakes
        else:
            if dataset != 'Deepfakes' and c_type == 'models':
                print('Models only available for Deepfakes. Aborting')
                return
            dataset_output_path = join(output_path, dataset_path, c_type)
            print('Output path: {}'.format(dataset_output_path))

            # Get Deepfakes models
            for folder in tqdm(filelist):
                folder_filelist = DEEPFAKES_MODEL_NAMES

                # Folder paths
                folder_base_url = args.deepfakes_model_url + folder + '/'
                folder_dataset_output_path = join(dataset_output_path,
                                                  folder)
                download_files(folder_filelist, folder_base_url,
                               folder_dataset_output_path,
                               report_progress=False)   # already done


if __name__ == "__main__":
    args = parse_args()
    main(args)"""

with open("download_ffpp.py", "w") as f:
    f.write(download_script)

In [ ]:
!pip install tqdm


In [ ]:
!python download_ffpp.py ffpp_data -d original -c c23 -t videos -n 500 --server EU2
!python download_ffpp.py ffpp_data -d Deepfakes -c c23 -t videos -n 500 --server EU2

By pressing any key to continue you confirm that you have agreed to the FaceForensics terms of use as described at:
http://kaldir.vc.in.tum.de/faceforensics/webpage/FaceForensics_TOS.pdf
***
Press any key to continue, or CTRL-C to exit.
a
Output path: ffpp_data/original_sequences/youtube/c23/videos
100% 500/500 [16:30<00:00,  1.98s/it]
By pressing any key to continue you confirm that you have agreed to the FaceForensics terms of use as described at:
http://kaldir.vc.in.tum.de/faceforensics/webpage/FaceForensics_TOS.pdf
***
Press any key to continue, or CTRL-C to exit.
h
Output path: ffpp_data/manipulated_sequences/Deepfakes/c23/videos
100% 500/500 [16:55<00:00,  2.03s/it]


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import shutil

# Specify the folder you want to upload
source_folder = '/content/ffpp_data'  # Path of the folder in Colab

# Specify the destination path in Google Drive
destination_folder = '/content/drive/My Drive/Deep Learning'  # Path in Google Drive

# Copy the folder to Google Drive
shutil.copytree(source_folder, destination_folder)


'/content/drive/My Drive/Deep Learning'

In [ ]:
import shutil

# Specify the folder you want to upload
source_folder = '/content/faces_temp'  # Path of the folder in Colab

# Specify the destination path in Google Drive
destination_folder = '/content/drive/My Drive/Deep Learning'  # Path in Google Drive

# Copy the folder to Google Drive
shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)
shutil.copytree('/content/frames_temp', destination_folder, dirs_exist_ok=True)

'/content/drive/My Drive/Deep Learning'

In [ ]:
pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00


In [ ]:
import cv2
import os
import shutil
import pandas as pd
from mtcnn import MTCNN

# Function to extract frames from a video
def extract_frames(video_path, output_dir, frame_skip=10):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved_count:03}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1
        frame_count += 1
    cap.release()

# Function to crop faces and assign labels
def crop_faces(input_dir, output_dir, video_id, label, label_list):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    detector = MTCNN()
    for img_file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        result = detector.detect_faces(img)
        if result:
            x, y, w, h = result[0]['box']
            face = img[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (128, 128))
            output_path = os.path.join(output_dir, f"{video_id}_{img_file}")
            cv2.imwrite(output_path, face_resized)
            label_list.append({'filename': f"{video_id}_{img_file}", 'label': label})


In [ ]:
video = '/content/drive/MyDrive/Deep Learning/manipulated_sequences/Deepfakes/c23/videos/004_982.mp4'
frame_dir = '/content/frames_temp'
face_dir = '/content/faces_temp'

# Extract every 10th frame
extract_frames(video, frame_dir, frame_skip=10)

# Crop faces from extracted frames
crop_faces(frame_dir, face_dir)

In [ ]:
import os
import pandas as pd

video_folder_real = '/content/drive/MyDrive/Deep Learning/original_sequences/youtube/c23/videos'
video_folder_fake = '/content/drive/MyDrive/Deep Learning/manipulated_sequences/Deepfakes/c23/videos'
base_output_dir = '/content/drive/MyDrive/Deep Learning/processed_faces'
label_data = []

# Function to check if the frames or cropped faces already exist
def check_if_processed(video_id, label):
    temp_frames = f'/content/drive/MyDrive/Deep Learning/temp_frames_{video_id}'
    output_faces = os.path.join(base_output_dir, 'real' if label == 0 else 'fake', video_id)

    # Check if the frames and cropped faces already exist
    if os.path.exists(output_faces):
        print(f"Video {video_id} already processed.")
        return True
    return False

# Process real videos
for video_name in os.listdir(video_folder_real):
    if video_name.endswith('.mp4'):
        video_id = os.path.splitext(video_name)[0]

        # Check if already processed
        if check_if_processed(video_id, label=0):
            continue

        video_path = os.path.join(video_folder_real, video_name)
        temp_frames = f'/content/drive/MyDrive/Deep Learning/temp_frames_{video_id}'
        output_faces = os.path.join(base_output_dir, 'real')
        extract_frames(video_path, temp_frames, frame_skip=10)
        crop_faces(temp_frames, output_faces, video_id, label=0, label_list=label_data)

# Process fake videos
for video_name in os.listdir(video_folder_fake):
    if video_name.endswith('.mp4'):
        video_id = os.path.splitext(video_name)[0]

        # Check if already processed
        if check_if_processed(video_id, label=1):
            continue

        video_path = os.path.join(video_folder_fake, video_name)
        temp_frames = f'/content/drive/MyDrive/Deep Learning/temp_frames_{video_id}'
        output_faces = os.path.join(base_output_dir, 'fake')
        extract_frames(video_path, temp_frames, frame_skip=10)
        crop_faces(temp_frames, output_faces, video_id, label=1, label_list=label_data)

# Save labels to CSV
df = pd.DataFrame(label_data)
df.to_csv('/content/drive/MyDrive/Deep Learning/face_labels.csv', index=False)
print("Labels saved to face_labels.csv")


KeyboardInterrupt: 

In [ ]:
import cv2
import os
import shutil
import pandas as pd
from mtcnn import MTCNN

# Function to extract frames from a video
def extract_frames(video_path, output_dir, frame_skip=10):
    if os.path.exists(output_dir):
        print(f"Frames already extracted for {video_path}, skipping extraction.")
        return  # Skip extraction if frames already exist
    os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved_count:03}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1
        frame_count += 1
    cap.release()

# Function to crop faces and assign labels
def crop_faces(input_dir, output_dir, video_id, label, label_list):
    if os.path.exists(output_dir):
        print(f"Cropped faces already saved for {video_id}, skipping face cropping.")
        return  # Skip face cropping if faces already exist
    os.makedirs(output_dir)

    detector = MTCNN()
    for img_file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        result = detector.detect_faces(img)
        if result:
            x, y, w, h = result[0]['box']
            face = img[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (128, 128))
            output_path = os.path.join(output_dir, f"{video_id}_{img_file}")
            cv2.imwrite(output_path, face_resized)
            label_list.append({'filename': f"{video_id}_{img_file}", 'label': label})

# Path setup
video_folder_real = '/content/drive/MyDrive/Deep Learning/original_sequences/youtube/c23/videos'
video_folder_fake = '/content/drive/MyDrive/Deep Learning/manipulated_sequences/Deepfakes/c23/videos'
base_output_dir = '/content/drive/MyDrive/Deep Learning/processed_faces'
label_data = []

# Process real videos
for video_name in os.listdir(video_folder_real):
    if video_name.endswith('.mp4'):
        video_id = os.path.splitext(video_name)[0]
        temp_frames = f'/content/drive/MyDrive/Deep Learning/temp_frames_{video_id}'
        output_faces = os.path.join(base_output_dir, 'real', video_id)

        # Check if frames already exist
        if os.path.exists(temp_frames):
            print(f"Frames already extracted for {video_id}, skipping extraction.")
        else:
            # Extract frames only if they don't already exist
            extract_frames(os.path.join(video_folder_real, video_name), temp_frames, frame_skip=10)

        # Check if cropped faces already exist
        if os.path.exists(output_faces):
            print(f"Cropped faces already saved for {video_id}, skipping face cropping.")
        else:
            # Crop faces only if they haven't been processed already
            crop_faces(temp_frames, output_faces, video_id, label=0, label_list=label_data)

# Process fake videos
for video_name in os.listdir(video_folder_fake):
    if video_name.endswith('.mp4'):
        video_id = os.path.splitext(video_name)[0]
        temp_frames = f'/content/drive/MyDrive/Deep Learning/temp_frames_{video_id}'
        output_faces = os.path.join(base_output_dir, 'fake', video_id)

        # Check if frames already exist
        if os.path.exists(temp_frames):
            print(f"Frames already extracted for {video_id}, skipping extraction.")
        else:
            # Extract frames only if they don't already exist
            extract_frames(os.path.join(video_folder_fake, video_name), temp_frames, frame_skip=10)

        # Check if cropped faces already exist
        if os.path.exists(output_faces):
            print(f"Cropped faces already saved for {video_id}, skipping face cropping.")
        else:
            # Crop faces only if they haven't been processed already
            crop_faces(temp_frames, output_faces, video_id, label=1, label_list=label_data)

# Save labels to CSV
df = pd.DataFrame(label_data)
df.to_csv('/content/drive/MyDrive/Deep Learning/face_labels.csv', index=False)
print("Labels saved to face_labels.csv")


Frames already extracted for 585, skipping extraction.
Frames already extracted for 599, skipping extraction.
Frames already extracted for 469, skipping extraction.
Frames already extracted for 481, skipping extraction.
Frames already extracted for 183, skipping extraction.
Frames already extracted for 253, skipping extraction.
Frames already extracted for 672, skipping extraction.
Frames already extracted for 720, skipping extraction.
Frames already extracted for 866, skipping extraction.
Frames already extracted for 878, skipping extraction.


KeyboardInterrupt: 

In [21]:
import cv2
import os
import shutil
import pandas as pd
from mtcnn import MTCNN

# Function to extract and process frames from a video
def extract_and_process_frames(video_path, output_dir, frame_skip=20, video_id=None, label=None, label_list=None):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Create output directory if it doesn't exist

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    detector = MTCNN()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:  # Save every 20th frame
            # Detect faces directly from the extracted frame
            result = detector.detect_faces(frame)
            if result:  # If faces are detected, crop and save
                for i, res in enumerate(result):
                    x, y, w, h = res['box']
                    face = frame[y:y+h, x:x+w]
                    face_resized = cv2.resize(face, (128, 128))  # Resize to 128x128
                    output_path = os.path.join(output_dir, f"{video_id}_frame_{saved_count:03}_face_{i}.jpg")
                    cv2.imwrite(output_path, face_resized)  # Save cropped face image
                    label_list.append({'filename': f"{video_id}_frame_{saved_count:03}_face_{i}.jpg", 'label': label})
                    saved_count += 1
        frame_count += 1

    cap.release()

# Path setup
video_folder_real = '/content/drive/MyDrive/Deep Learning/original_sequences/youtube/c23/videos'
video_folder_fake = '/content/drive/MyDrive/Deep Learning/manipulated_sequences/Deepfakes/c23/videos'
base_output_dir = '/content/drive/MyDrive/Deep Learning/processed_faces'
label_data = []

# Process first 150 real videos
real_videos = [video for video in os.listdir(video_folder_real) if video.endswith('.mp4')][:150]
for video_name in real_videos:
    video_id = os.path.splitext(video_name)[0]
    output_faces = os.path.join(base_output_dir, 'real', video_id)

    # Check if cropped faces already exist
    if os.path.exists(output_faces) and os.listdir(output_faces):  # If faces already cropped
        print(f"Cropped faces already saved for {video_id}, skipping face cropping.")
    else:
        # Extract and crop faces directly into processed_faces
        extract_and_process_frames(
            os.path.join(video_folder_real, video_name),
            output_faces,
            frame_skip=20,  # Save every 20th frame
            video_id=video_id,
            label=0,  # Label for real videos
            label_list=label_data
        )

# Process first 150 fake videos
fake_videos = [video for video in os.listdir(video_folder_fake) if video.endswith('.mp4')][:150]
for video_name in fake_videos:
    video_id = os.path.splitext(video_name)[0]
    output_faces = os.path.join(base_output_dir, 'fake', video_id)

    # Check if cropped faces already exist
    if os.path.exists(output_faces) and os.listdir(output_faces):  # If faces already cropped
        print(f"Cropped faces already saved for {video_id}, skipping face cropping.")
    else:
        # Extract and crop faces directly into processed_faces
        extract_and_process_frames(
            os.path.join(video_folder_fake, video_name),
            output_faces,
            frame_skip=20,  # Save every 20th frame
            video_id=video_id,
            label=1,  # Label for fake videos
            label_list=label_data
        )

# Save labels to CSV
df = pd.DataFrame(label_data)
df.to_csv('/content/drive/MyDrive/Deep Learning/face_labels.csv', index=False)
print("Labels saved to face_labels.csv")


Labels saved to face_labels.csv


In [22]:
# Read the CSV into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Deep Learning/face_labels.csv')

# Count the occurrences of each label (0 and 1)
label_counts = df['label'].value_counts()

# Print the counts for labels 0 and 1
print(f"Number of '0' labels frames: {label_counts.get(0, 0)}")
print(f"Number of '1' labels frames: {label_counts.get(1, 0)}")

Number of '0' labels frames: 4800
Number of '1' labels frames: 4752


In [ ]:
!python --version


Python 3.11.12
